<a href="https://colab.research.google.com/github/adolfoguimaraes/Antlr4-Project/blob/main/Estatistica_Parte2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalação, Configurações e Imports

In [1]:
# Instalação do PySpark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Imports
from google.colab import drive
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [3]:
# Diretório das bases 
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/datasets/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Inicialização do Spark
spark = (SparkSession
         .builder
         .appName("Stats App")
         .getOrCreate())
     

## Base de Dados

https://nijianmo.github.io/amazon/index.html

Vamos carregar nossa base de dados no formato JSON.

Com o arquivo Movies_and_TV_5.json. 

Vamos criar um pipeline de processamento de texto e exibição de algumas estatísticas e depois aplicar a diferentes cenários.

In [5]:
shows = (spark
         .read
         .json(data_path + "tv_shows/Movies_and_TV_5.json"))

In [6]:
shows.count()

3410019

In [7]:
shows.printSchema()

root
 |-- asin: string (nullable = true)
 |-- image: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- style: struct (nullable = true)
 |    |-- Format:: string (nullable = true)
 |    |-- Shape:: string (nullable = true)
 |    |-- Size:: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)
 |-- verified: boolean (nullable = true)
 |-- vote: string (nullable = true)



In [8]:
my_data = (shows
           .select('asin','overall','reviewText','unixReviewTime','reviewerID')
           .withColumn("reviewDate",
                       F.date_format(
                           F.from_unixtime(F.col("unixReviewTime")),"yyyy-MM-dd"))
           .drop(F.col('unixReviewTime'))
)
my_data.show(5)

+----------+-------+--------------------+--------------+----------+
|      asin|overall|          reviewText|    reviewerID|reviewDate|
+----------+-------+--------------------+--------------+----------+
|0005089549|    5.0|So sorry I didn't...|A2M1CU2IRZG0K9|2012-11-09|
|0005089549|    5.0|Believe me when I...| AFTUJYISOFHY6|2011-12-30|
|000503860X|    5.0|I have seen X liv...|A3JVF9Y53BEOGC|2005-04-21|
|000503860X|    5.0|I was so excited ...|A12VPEOEZS1KTC|2005-04-06|
|000503860X|    5.0|X is one of the b...| ATLZNVLYKP9AZ|2010-12-03|
+----------+-------+--------------------+--------------+----------+
only showing top 5 rows



In [9]:
my_data.groupBy("asin").agg(F.count("asin")).count()

60175

In [10]:
my_data.groupBy("reviewerID").agg(F.count("reviewerID")).count()

297529

O DataFrame `my_data` vai ser nosso ponto de partida. Inicialmente, queremos responder as seguintes perguntas: 

* Qual a média de notas para todos os produtos, por produto e por cliente? É uma boa métrica, nesse caso?  
* Qual o total de avaliações por produto e por cliente?

In [11]:
my_data.select(F.col("overall")).summary().show()

+-------+------------------+
|summary|           overall|
+-------+------------------+
|  count|           3410019|
|   mean| 4.221320174462371|
| stddev|1.1664558754885523|
|    min|               1.0|
|    25%|               4.0|
|    50%|               5.0|
|    75%|               5.0|
|    max|               5.0|
+-------+------------------+



In [12]:
(my_data
  .select("asin","overall")
  .groupBy("asin")
  .agg(
      F.mean("overall").alias("media"),
      F.percentile_approx("overall", 0.5).alias("mediana"),
      F.count("asin").alias("total_avaliacoes")
      )
  .show())

+----------+------------------+-------+----------------+
|      asin|             media|mediana|total_avaliacoes|
+----------+------------------+-------+----------------+
|0000143588|               5.0|    5.0|               5|
|0001526863| 4.928571428571429|    5.0|              14|
|0005000009| 4.046511627906977|    5.0|              43|
|0005019281| 4.425742574257426|    5.0|             303|
|000503860X| 4.666666666666667|    5.0|               9|
|0005092663|            4.1875|    5.0|              16|
|0005114349| 4.509803921568627|    5.0|              51|
|0005119367| 4.786666666666667|    5.0|             225|
|0006486576| 4.418181818181818|    5.0|              55|
|0307142477|2.7142857142857144|    2.0|               7|
|0615115187|4.7407407407407405|    5.0|              27|
|073891908X|               3.4|    5.0|               5|
|0738920061|3.8333333333333335|    4.0|               6|
|0739040375|             4.625|    5.0|               8|
|0740318764| 4.909090909090909|

In [13]:
(my_data
  .select("reviewerID","overall")
  .groupBy("reviewerID")
  .agg(
      F.mean("overall").alias("media"),
      F.percentile_approx("overall", 0.5).alias("mediana"),
      F.count("reviewerID").alias("total_avaliacoes")
      )
  .show())

+--------------------+------------------+-------+----------------+
|          reviewerID|             media|mediana|total_avaliacoes|
+--------------------+------------------+-------+----------------+
|A0009988MRFQ3TROTQPI|               5.0|    5.0|               8|
|A00311542N70JGNHUZPI|               5.0|    5.0|               5|
|A0040548BPHKXMHH3NTI|3.8461538461538463|    4.0|              13|
|A0045750NMY09KVSTE01|               5.0|    5.0|              10|
|A0048168OBFNFN7WW8XC|               5.0|    5.0|               5|
|A0070287EH1P1FHWO3GW|               4.6|    5.0|              10|
|A0072041HVZ3465DXUOR|               5.0|    5.0|               7|
|A0149220O8WD7KG3O5XR|               4.5|    5.0|              18|
|A0264020562YYZ05Y2R5| 4.571428571428571|    5.0|               7|
|A0275782DY4RT0D8KADG|               5.0|    5.0|               7|
|A0293382VV3VQQ607PY4|               4.0|    4.0|               9|
|A0322174KPHFYVAJWTR2| 4.888888888888889|    5.0|             

Vamos para mais perguntas: 

* Quais palavras caracterizam cada produto de acordo com o review dado? 

In [14]:
spark.version

'3.3.2'

In [15]:
text_data = my_data.select("asin","reviewText")

text_data = (text_data.select("asin", 
                              F.explode(F.lower('reviewText'))))

text_data.show()

AnalysisException: ignored

In [ ]:
episodes_name.select(
    F.explode("name").alias("name")
).show(5, False)